In [2]:
import pandas as pd
import numpy as np
import tensorflow as tf
import keras
import matplotlib.pyplot as plt
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
import efficientnet.keras as efn 
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.vgg16 import preprocess_input
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.layers import Input
from sklearn.metrics import confusion_matrix

ModuleNotFoundError: No module named 'tensorflow'

In [ ]:
from tensorflow.keras.layers import Activation, Dense

In [ ]:
def get_data_generator(train_data_path, val_data_path, targetsize, classmode, batchsize):

    train_datagen = ImageDataGenerator(preprocessing_function=preprocess_input)
    val_datagen = ImageDataGenerator(preprocessing_function=preprocess_input)
    
    train_generator = train_datagen.flow_from_directory(train_data_path,
                                                        target_size=targetsize,
                                                        color_mode='rgb',
                                                        batch_size=batchsize,
                                                        class_mode=classmode,
                                                        shuffle=True)
    
    val_generator = val_datagen.flow_from_directory(val_data_path,
                                                    target_size=targetsize,
                                                    color_mode='rgb',
                                                    batch_size=batchsize,
                                                    class_mode=classmode,
                                                    shuffle=False)
 
    return train_generator, val_generator   
    

def train_evaluate_the_model(train_generator, val_generator, optimizer, epochs, dropout_value, TheModel):

    # Freeze base layers for training
    for layer in TheModel.layers:
        layer.trainable = False
    
    # load pretrained model and add layers on top of the model
    x = TheModel.output
    x = tf.keras.layers.Flatten()(x)
    x = Dense(1024, activation="relu")(x)
    x = Dense(1024, activation="relu")(x)
    x = Dense(512, activation="relu")(x)
    x = tf.keras.layers.Dropout(dropout_value)(x)
    predictions = Dense(4, activation="sigmoid")(x)
    model = Model(inputs=TheModel.input, outputs=predictions)
    
    # Compile the model
    model.compile(optimizer=optimizer,
                  loss='categorical_crossentropy',
                  metrics=['accuracy'])
    
    step_size_train = train_generator.n // train_generator.batch_size
    
    # Fit the model
    r = model.fit_generator(generator=train_generator,
                            steps_per_epoch=step_size_train,
                            epochs=epochs,
                            validation_data=val_generator)
    
    # Evaluate the model
    scores = model.evaluate(val_generator)
    accuracy = scores[1] * 100
    
    return accuracy, model
